In [0]:
pip install pandas openpyxl 

In [0]:
%restart_python

In [0]:
# MAGIC %md
# MAGIC # 🔄 Sincronizador de Reglas (Excel -> Catálogo de Validaciones, metadatos de tablas y catálogo de reglas)
# MAGIC 
# MAGIC Este notebook es la herramienta para poblar 'dq_tables_config', 'dq_rules_library' y 'dq_validations_catalog'
# MAGIC leyendo la plantilla de Excel validationsConfig
# MAGIC 
# MAGIC **Funciones:**
# MAGIC 1.  'load_and_validate_excel': lee y valida el fichero Excel
# MAGIC 2.  'build_json_definition': construye el JSON 'rule_definition' a partir de las columnas de parámetros
# MAGIC 2.  'sync_table_config': sincroniza los cambios en 'dq_tables_config' con la función 'MERGE'
# MAGIC 2.  'sync_rule_library': sincroniza los cambios en 'dq_rules_library' con la función 'MERGE'
# MAGIC 2.  'sync_validations_catalog': sincroniza los cambios en 'dq_validations_catalog' con la función 'MERGE'
# MAGIC 2.  'main': orquesta las llamadas a las demás funciones

# COMMAND ----------
# Libreria para poder leer el .xlsx por medio de Pandas. 
# !!!!!!!!!!!!!!!comentario interno (borrar cuando se solucione): O se instala en el cluster o se añade a otra linea de código (si falla de esta forma)!!!!!!!!!!!!!!!
# MAGIC %pip install pandas openpyxl

# COMMAND ----------

# DBTITLE 1, 1. Imports, constantes y widgets
import pandas as pd
import json
import uuid
from datetime import datetime
from pyspark.sql.functions import col, lit, current_timestamp, udf, struct, split
from pyspark.sql.types import StringType, MapType, StructType, StructField, BooleanType, ArrayType
from delta.tables import DeltaTable

# --- Constantes de tablas ---
dbutils.widgets.text("catalog_name", "workspace", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("schema_name", "dq_framework", "Esquema de UC donde residen las tablas")
dbutils.widgets.text("tables_sheet", "Tablas", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("rules_sheet", "Reglas", "Esquema de UC donde residen las tablas")
dbutils.widgets.text("validations_sheet", "Validaciones", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("excel_file_path", "/Volumes/workspace/dq_framework/template/configValidaciones.xlsx", "Ruta al fichero Excel con la información")

CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")

TABLE_CONFIG = f"{CATALOG}.{SCHEMA}.dq_tables_config"
RULE_LIB_TABLE = f"{CATALOG}.{SCHEMA}.dq_rules_library"
CATALOG_TABLE = f"{CATALOG}.{SCHEMA}.dq_validations_catalog"

CONFIG_SHEET = dbutils.widgets.get("tables_sheet")
LIBRARY_SHEET = dbutils.widgets.get("rules_sheet")
VALIDATIONS_SHEET = dbutils.widgets.get("validations_sheet")
EXCEL_PATH = dbutils.widgets.get("excel_file_path")

# --- Columnas del Excel ---

CONFIG_EXPECTED_COLS = [
    'Id_tabla', 'Nombre_tabla', 'Nombre_técnico', 
    'Clave_primaria', 'Tabla_staging_evidencias'
]

LIBRARY_EXPECTED_COLS = [
    'Id_regla', 'Nombre_técnico', 'Nombre_funcional','Descripción', 'Nivel_regla',
    'Dimensión_reglas', 'Tipo_implementación', 'Etiquetas', 'Clase', 
    'Propietario', 'Fecha_actualizacion', 'Actualizado_por'
]

VALIDATIONS_EXPECTED_COLS = [
    'Id_validación','Id_regla', 'Id_tabla', 'Nombre_técnico', 'Dominio_tabla', 'Definición_perimetro',
    'Param_columnas', 'Param_valor','Param_valor_min','Param_valor_max', 'Param_conjunto',
    'Param_tipo_dato','Param_expresion', 'Param_query_SQL', 'Param_merge_columnas', 
    'Severidad','Owner','Etiquetas', 'Actualizado_por', 'Fecha_actualización', 'Validación_activa'
]

# COMMAND ----------

# DBTITLE 2, 2. Leer y validar Excel
def load_and_validate_excel(excel_path, sheet_name, expected_cols):
    """
    Lee el fichero Excel usando Pandas y lo convierte a un DataFrame de Spark
    Valida que las columnas esperadas existan.
    """
    try:
        # Uso de Pandas para leer, forzando todo a string para evitar errores de tipo
        pandas_df = pd.read_excel(excel_path, sheet_name=sheet_name, dtype=str).fillna('')

        # Validar que las columnas necesarias existen      
        missing_cols = [col for col in expected_cols if col not in pandas_df.columns]
        if missing_cols:
            raise ValueError(f"Faltan columnas obligatorias en el Excel: {missing_cols}")
        
        pandas_df = pandas_df.dropna(how='all')
        if pandas_df.empty:
            print(f"Hoja '{sheet_name}' está vacía. Saltando sincronización.")
            return None
        
        # Convertir el DataFrame de Pandas a un DataFrame de Spark
        return spark.createDataFrame(pandas_df)

    except FileNotFoundError:
        print(f"Error Crítico: No se encontró el fichero Excel en {excel_path}")
        raise
    except ValueError as ve:
        print(f"Error de Validación: {ve}")
        raise
    except Exception as e:
        errors_list.append({"sheet": sheet_name, "fila": None, "error": str(e)})
        return None, errors_list
    
# COMMAND ----------
            
# COMMAND ----------
# DBTITLE 3, 3. Función Python para construir el JSON (la lógica de la regla)
def build_json_definition(technical_name, rule_type, param_columnas_str, param_valor, param_min, param_max, param_conjunto_str, param_sql, param_tipo, param_query, param_merge):
    """
    Función: con las columnas de parámetros del Excel
    construye el string JSON 'rule_definition' que espera el orquestador.
    """
    
    # --- Diccionario base ---
    params_dict = {}
    
    try:
        # --- Lógica para reglas BUILT-IN ---
        if rule_type == 'BUILT-IN':
            
            # Reglas que solo usan 'Param_Columnas'
            if technical_name in ["is_not_null", "is_not_empty", "is_not_null_and_not_empty", "is_not_in_future"]:
                columns = [col.strip() for col in param_columnas_str.split(',') if col and col.strip()]
                if columns: params_dict["column"] = columns[0]
            
            elif technical_name == "is_unique":
                columns = [col.strip() for col in param_columnas_str.split(',') if col and col.strip()]
                if columns: params_dict["columns"] = columns # Espera 'columns' (plural)

            # Regla 'is_in_list' y 'is_not_null_and_is_in_list'
            elif technical_name in ["is_in_list", "is_not_null_and_is_in_list"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                allowed_values = [val.strip() for val in param_conjunto_str.split(',') if val and val.strip()]
                if allowed_values: params_dict["allowed"] = allowed_values
            
            # Regla 'regex_match'
            elif technical_name == "regex_match":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["regex"] = param_valor
             
            # Regla 'is_equal_to' y 'is_not_equal_to'
            elif technical_name in ["is_equal_to", "is_not_equal_to"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["value"] = param_valor

            # Regla 'is_valid_date'
            elif technical_name == "is_valid_date":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["date_format"] = param_valor

            # Regla 'is_valid_timestamp'
            elif technical_name == "is_valid_timestamp":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                if param_valor: params_dict["timestamp_format"] = param_valor
                
            # Regla 'is_in_range' y 'is_not_in_range'
            elif technical_name in ["is_in_range", "is_not_in_range"]:
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                try: 
                    if param_min: params_dict["min_limit"] = float(param_min)
                except (ValueError, TypeError): params_dict["min_limit"] = param_min
                try: 
                    if param_max: params_dict["max_limit"] = float(param_max)
                except (ValueError, TypeError): params_dict["max_limit"] = param_max
            
            # Regla 'is_not_less_than'
            elif technical_name == "is_not_less_than":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                try: 
                    if param_min: params_dict["limit"] = float(param_min)
                except (ValueError, TypeError): params_dict["limit"] = param_min
            
            # Regla 'is_not_greater_than'
            elif technical_name == "is_not_greater_than":
                col_name = param_columnas_str.strip() if param_columnas_str else None
                if col_name: params_dict["column"] = col_name
                try: 
                    if param_max: params_dict["limit"] = float(param_max)
                except (ValueError, TypeError): params_dict["limit"] = param_max

            # Regla 'sql_expression'
            elif technical_name == "sql_expression":
                if param_sql: params_dict["expression"] = param_sql
            
             # Regla 'sql_query'
            elif technical_name == "sql_query":
                if param_query: params_dict["query"] = param_query
                merge_cols = [col.strip() for col in param_merge.split(',') if col and col.strip()]
                if merge_cols: params_dict["merge_columns"] = merge_cols

        # --- Lógica para Reglas CUSTOM ---
        elif rule_type in ['CUSTOM']:
             columns = [col.strip() for col in param_columnas_str.split(',') if col and col.strip()]
             if columns: params_dict["columns"] = columns

        # Convertir el diccionario de parámetros final a un string JSON
        return json.dumps(params_dict)

    except Exception as e:
        return json.dumps({"error": f"Error al construir JSON de params: {e}"})
    
# COMMAND ----------

# DBTITLE 5, 5. Sincronizar dq_rule_library
def sync_rule_library(df_excel, target_table_name):
    """
    Toma el DataFrame de la hoja 'Reglas' y lo sincroniza
    con la tabla Delta 'dq_rules_library'.
    """
    if df_excel is None:
        return 0, 0
        
    try:
        df_source = df_excel.select(
            col("Id_regla").alias("rule_id"),
            col("Nombre_técnico").alias("technical_rule_name"),
            col("Nombre_funcional").alias("functional_name"),
            col("Descripción").alias("description"),
            col("Nivel_regla").alias("rule_level"),
            col("Dimensión_reglas").alias("dimension_dq"),
            col("Tipo_implementación").alias("implementation_type"),
            current_timestamp().alias("updated_at"),
            col("Actualizado_por").alias("updated_by"),
            split(col("Etiquetas"), ",\s*").alias("tags"),
            col("Propietario").alias("owner"),
        ).filter(col("rule_id") != "")

        delta_table = DeltaTable.forName(spark, target_table_name)

        (delta_table.alias("target")
         .merge(
             df_source.alias("source"),
             condition = "target.rule_id = source.rule_id"
         )
         .whenMatchedUpdate(set = {
             "technical_rule_name": "source.technical_rule_name",
             "functional_name": "source.functional_name",
             "description": "source.description",
             "rule_level": "source.rule_level",
             "dimension_dq": "source.dimension_dq",
             "implementation_type": "source.implementation_type",
             "updated_at": "source.updated_at",
             "updated_by": "source.updated_by",
             "tags": "source.tags",
             "owner": "source.owner"
         })
         .whenNotMatchedInsert(values = {
             "rule_id": "source.rule_id",
             "technical_rule_name": "source.technical_rule_name",
             "functional_name": "source.functional_name",
             "description": "source.description",
             "rule_level": "source.rule_level",
             "dimension_dq": "source.dimension_dq",
             "implementation_type": "source.implementation_type",
             "updated_at": "source.updated_at",
             "updated_by": "source.updated_by",
             "tags": "source.tags",
             "owner": "source.owner"
         })
         .execute()
        )
        count = df_source.count()
        print(f"Éxito: {count} registros sincronizados en {target_table_name}.")
        return count

    except Exception as e:
        print(f"ERROR al sincronizar {target_table_name}: {e}")
        raise

build_json_udf = udf(build_json_definition, StringType())

# DBTITLE 4,4. Función genérica para sincronizar Delta por hoja
def sync_delta(df_spark, target_table_name, merge_keys, column_mapping, sheet_name):
    if df_spark is None:
        return 0, []
    errors_list = []
    try:
        delta_table = DeltaTable.forName(spark, target_table_name)
        delta_table.alias("target").merge(
            df_spark.alias("source"),
            " AND ".join([f"target.{k} = source.{k}" for k in merge_keys])
        ).whenMatchedUpdate(set={k:v for k,v in column_mapping.items()}) \
         .whenNotMatchedInsert(values={k:v for k,v in column_mapping.items()}) \
         .execute()
        return df_spark.count(), errors_list
    except Exception as e:
        errors_list.append({"sheet": sheet_name, "fila": None, "error": str(e)})
        return 0, errors_list

# DBTITLE 5,5. Sincronización específica por hoja
def sync_tables(df_excel):

    if df_excel is None:
        return 0, 0
    
    df_valid = df_excel.select(
        col("Id_tabla").alias("table_id"),
        col("Nombre_tabla").alias("table_name"),
        col("Nombre_técnico").alias("table_name_tech"),
        col("Clave_primaria").alias("primary_key"),
        col("Tabla_staging_evidencias").alias("staging_evidences_table")
    ).filter(col("Id_tabla") != "")

    mapping = {c:c for c in df_valid.columns}
    return sync_delta(df_valid, TABLE_CONFIG, ["table_id"], mapping, "Tablas")   

def sync_rules(df_excel):

    if df_excel is None:
        return 0, 0
    
    df_valid = df_excel.select(
        col("Id_regla").alias("rule_id"),
        col("Nombre_técnico").alias("technical_rule_name"),
        col("Nombre_funcional").alias("functional_name"),
        col("Descripción").alias("description"),
        col("Nivel_regla").alias("rule_level"),
        col("Dimensión_reglas").alias("dimension_dq"),
        col("Tipo_implementación").alias("implementation_type"),
        current_timestamp().alias("updated_at"),
        col("Actualizado_por").alias("updated_by"),
        split(col("Etiquetas"), ",\s*").alias("tags"),
        col("Propietario").alias("owner")
    ).filter(col("Id_regla") != "")
    mapping = {c:c for c in df_valid.columns}
    return sync_delta(df_valid, RULE_LIB_TABLE, ["rule_id"], mapping, "Reglas")

def sync_validations(df_excel):

    if df_excel is None:
        return 0, 0
    
    df_valid = df_excel.select(
        col("Id_validación").alias("validation_id"),
        col("Id_regla").alias("rule_id"),
        col("Id_tabla").alias("table_id"),
        col("Definición_perimetro").alias("perimeter_definition"),
        col("Validación_activa").cast(BooleanType()).alias("is_active"),
        col("Severidad").alias("severity"),
        build_json_udf(
            col("Nombre_técnico"), col("Tipo_regla"),
            col("Param_columnas"), col("Param_valor"),
            col("Param_valor_min"), col("Param_valor_max"),
            col("Param_conjunto"), col("Param_expresion"),
            col("Param_tipo_dato"), col("Param_query_SQL"),
            col("Param_merge_columnas")
        ).alias("validation_definition"),
        col("Dominio_tabla").alias("domain"),
        split(col("Param_columnas"), ",\s*").alias("Param_columns"),
        col("Param_valor").alias("Param_value"),
        col("Param_valor_min").alias("Param_value_min"),
        col("Param_valor_max").alias("Param_value_max"),
        col("Param_conjunto").alias("Param_range"),
        col("Param_expresion").alias("Param_expression"),
        col("Param_tipo_dato").alias("Param_data_type"),
        col("Param_query_SQL").alias("Param_query_SQL"),
        col("Param_merge_columnas").alias("Param_merge_columns"),
        col("Owner").alias("owner"),
        current_timestamp().alias("updated_at"),
        col("Actualizado_por").alias("updated_by"),
        split(col("Etiquetas"), ",\s*").alias("tags")
    ).filter(col("Id_validación") != "")
    mapping = {c:c for c in df_valid.columns}
    return sync_delta(df_valid, CATALOG_TABLE, ["validation_id"], mapping, "Validaciones")


# COMMAND ----------

# DBTITLE 7, 7. Función Principal (main) - Orquestación del script
def main():
    """
    Flujo principal del script de sincronización.
    """
    try:
        # --- 1. Sincronizar dq_tables_config ---
        df_tablas, errors_tablas = load_and_validate_excel(EXCEL_PATH, CONFIG_SHEET, CONFIG_EXPECTED_COLS)
        total_errors.extend(errors_tablas)
        synced_tablas, _ = sync_tables(df_tablas)
        
        # 2. Sincronizar Rules_library
        df_rules, errors_rules = load_and_validate_excel(EXCEL_PATH, LIBRARY_SHEET, LIBRARY_EXPECTED_COLS)
        total_errors.extend(errors_rules)
        synced_rules, _ = sync_rules(df_rules)
        
        # 3. Sincronizar Validations_catalog
        df_valids, errors_valids = load_and_validate_excel(EXCEL_PATH, VALIDATIONS_SHEET, VALIDATIONS_EXPECTED_COLS)
        total_errors.extend(errors_valids)
        synced_valids, _ = sync_validations(df_valids)
        
        print(f"Sincronización completa")
        if total_errors:
            print(f"Se detectaron {len(total_errors)} errores durante la sincronización:")
            #for e in total_errors: print(e)
        return f"Éxito: Tablas {synced_tablas}, Reglas {synced_rules}, Validaciones {synced_valids}, Errores {len(total_errors)}"
    except Exception as e:
        print(f"Fallo crítico: {e}")
        raise

# COMMAND ----------

# DBTITLE 7, 7. Punto de Entrada de Ejecución
if __name__ == "__main__":
    result_msg = main()
    dbutils.notebook.exit(result_msg)